In [2]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [3]:
df = pd.read_csv('home_data_kaggle\\train.csv').drop(['Alley','MasVnrType','FireplaceQu','PoolQC','Fence','MiscFeature'],axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 75 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [4]:
y= df.SalePrice
X = df.drop('SalePrice',axis=1)

X_train_full,X_val_full,y_train,y_val = train_test_split(X,y,test_size=0.2,train_size=0.8,random_state=0)

cat_cols = [col for col in X_train_full.columns if X_train_full[col].nunique()<10 and X_train_full[col].dtype == 'object']
num_cols = [col for col in X_train_full.columns if X_train_full[col].dtype in['int64','float64']]

full_cols = cat_cols+num_cols

X_train = X_train_full[full_cols].copy()
X_val = X_val_full[full_cols].copy()

In [5]:
X_val.info()

<class 'pandas.core.frame.DataFrame'>
Index: 292 entries, 529 to 61
Data columns (total 71 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSZoning       292 non-null    object 
 1   Street         292 non-null    object 
 2   LotShape       292 non-null    object 
 3   LandContour    292 non-null    object 
 4   Utilities      292 non-null    object 
 5   LotConfig      292 non-null    object 
 6   LandSlope      292 non-null    object 
 7   Condition1     292 non-null    object 
 8   Condition2     292 non-null    object 
 9   BldgType       292 non-null    object 
 10  HouseStyle     292 non-null    object 
 11  RoofStyle      292 non-null    object 
 12  RoofMatl       292 non-null    object 
 13  ExterQual      292 non-null    object 
 14  ExterCond      292 non-null    object 
 15  Foundation     292 non-null    object 
 16  BsmtQual       283 non-null    object 
 17  BsmtCond       283 non-null    object 
 18  BsmtExposure  

In [6]:
#Filling missing values in number columns
my_imputer = SimpleImputer()

#Filling missing values and encoding tex columns

text_processor = Pipeline(steps=[('text_imputer',SimpleImputer(strategy='most_frequent')),
                                ('text_encoder',OneHotEncoder(handle_unknown='ignore'))])


In [7]:
col_transformer = ColumnTransformer(transformers=[
    ('num_transformer',my_imputer,num_cols),
    ('text_transformer',text_processor,cat_cols)
])

In [8]:
X_val.info()

<class 'pandas.core.frame.DataFrame'>
Index: 292 entries, 529 to 61
Data columns (total 71 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSZoning       292 non-null    object 
 1   Street         292 non-null    object 
 2   LotShape       292 non-null    object 
 3   LandContour    292 non-null    object 
 4   Utilities      292 non-null    object 
 5   LotConfig      292 non-null    object 
 6   LandSlope      292 non-null    object 
 7   Condition1     292 non-null    object 
 8   Condition2     292 non-null    object 
 9   BldgType       292 non-null    object 
 10  HouseStyle     292 non-null    object 
 11  RoofStyle      292 non-null    object 
 12  RoofMatl       292 non-null    object 
 13  ExterQual      292 non-null    object 
 14  ExterCond      292 non-null    object 
 15  Foundation     292 non-null    object 
 16  BsmtQual       283 non-null    object 
 17  BsmtCond       283 non-null    object 
 18  BsmtExposure  

In [9]:
my_model = RandomForestRegressor(max_leaf_nodes=500,n_estimators=100)

main_pipe = Pipeline(steps=[('preprocessor',col_transformer),
                           ('modeler',my_model)])
main_pipe.fit(X_train,y_train)
preds = main_pipe.predict(X_val)

mea = mean_absolute_error(y_val,preds)
mea

                

np.float64(17466.90064728033)

In [10]:
from sklearn.model_selection import cross_val_score

score = -1 * cross_val_score(main_pipe,X,y,cv=3,scoring='neg_mean_absolute_error')
score.mean()

np.float64(18022.993291892028)

In [13]:
from xgboost import XGBRegressor

ModuleNotFoundError: No module named 'xgboost'